In [ ]:
# ! nvidia-smi

In [ ]:
# ! pip install captum timm shap grad-cam  albumentations

In [ ]:
import urllib

import matplotlib.pyplot as plt
import numpy as np
import timm
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from matplotlib.colors import LinearSegmentedColormap
from PIL import Image

In [ ]:
device = torch.device("cuda")
print(device)
model = timm.create_model("resnet18", pretrained=True)
model.eval()
model = model.to(device)

In [ ]:
# Download human-readable labels for ImageNet and get the classnames
url, filename = (
    "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt",
    "imagenet_classes.txt",
)
urllib.request.urlretrieve(url, filename)
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [ ]:
! wget -O cat.jpeg "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSKr5wT7rfkjkGvNeqgXjBmarC5ZNoZs-H2uMpML8O7Q4F9W-IlUQibBT6IPqyvX45NOgw&usqp=CAU"

In [ ]:
transform = T.Compose([T.Resize(224), T.ToTensor()])

transform_normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

img = Image.open("cat.jpeg")

transformed_img = transform(img)

img_tensor = transform_normalize(transformed_img)
img_tensor = img_tensor.unsqueeze(0)

In [ ]:
img_tensor = img_tensor.to(device)
output = model(img_tensor)
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)

pred_label_idx.squeeze_()
predicted_label = categories[pred_label_idx.item()]
print("Predicted:", predicted_label, "(", prediction_score.squeeze().item(), ")")

In [ ]:
pred_label_idx

In [ ]:
from captum.attr import GradientShap, IntegratedGradients, NoiseTunnel, Occlusion
from captum.attr import visualization as viz

In [ ]:
integrated_gradients = IntegratedGradients(model)
attributions_ig = integrated_gradients.attribute(img_tensor, target=pred_label_idx, n_steps=200)

In [ ]:
default_cmap = LinearSegmentedColormap.from_list(
    "custom blue", [(0, "#ffffff"), (0.25, "#000000"), (1, "#000000")], N=256
)

_ = viz.visualize_image_attr(
    np.transpose(attributions_ig.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    method="heat_map",
    cmap=default_cmap,
    show_colorbar=True,
    sign="positive",
    outlier_perc=1,
)

In [ ]:
noise_tunnel = NoiseTunnel(integrated_gradients)

attributions_ig_nt = noise_tunnel.attribute(
    img_tensor, nt_samples=10, nt_type="smoothgrad_sq", target=pred_label_idx
)
_ = viz.visualize_image_attr_multiple(
    np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    ["original_image", "heat_map"],
    ["all", "positive"],
    cmap=default_cmap,
    show_colorbar=True,
)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

gradient_shap = GradientShap(model)

# Defining baseline distribution of images
rand_img_dist = torch.cat([img_tensor * 0, img_tensor * 1])

attributions_gs = gradient_shap.attribute(
    img_tensor, n_samples=50, stdevs=0.0001, baselines=rand_img_dist, target=pred_label_idx
)
_ = viz.visualize_image_attr_multiple(
    np.transpose(attributions_gs.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    ["original_image", "heat_map"],
    ["all", "absolute_value"],
    cmap=default_cmap,
    show_colorbar=True,
)

In [ ]:
occlusion = Occlusion(model)

attributions_occ = occlusion.attribute(
    img_tensor,
    strides=(3, 8, 8),
    target=pred_label_idx,
    sliding_window_shapes=(3, 15, 15),
    baselines=0,
)

In [ ]:
_ = viz.visualize_image_attr_multiple(
    np.transpose(attributions_occ.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    np.transpose(transformed_img.squeeze().cpu().detach().numpy(), (1, 2, 0)),
    ["original_image", "heat_map"],
    ["all", "positive"],
    show_colorbar=True,
    outlier_perc=2,
)

In [ ]:
from captum.attr import DeepLift, IntegratedGradients, NoiseTunnel, Saliency
from captum.attr import visualization as viz

In [ ]:
transform = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

img = Image.open("cat.jpeg")

img_tensor = transform(img)
img_tensor = img_tensor.unsqueeze(0)
img_tensor.requires_grad = True
img_tensor = img_tensor.to(device)

In [ ]:
img_tensor.requires_grad

In [ ]:
saliency = Saliency(model)
grads = saliency.attribute(img_tensor, target=285)
grads = np.transpose(grads.squeeze().cpu().detach().numpy(), (1, 2, 0))

In [ ]:
original_image = np.transpose((img_tensor.squeeze(0).cpu().detach().numpy() / 2) + 0.5, (1, 2, 0))

In [ ]:
original_image.shape

In [ ]:
_ = viz.visualize_image_attr(None, original_image, method="original_image", title="Original Image")

In [ ]:
_ = viz.visualize_image_attr(
    grads,
    original_image,
    method="blended_heat_map",
    sign="absolute_value",
    show_colorbar=True,
    title="Overlayed Gradient Magnitudes",
)

In [ ]:
from pytorch_grad_cam import (
    AblationCAM,
    EigenCAM,
    FullGrad,
    GradCAM,
    GradCAMPlusPlus,
    HiResCAM,
    ScoreCAM,
    XGradCAM,
)
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [ ]:
target_layers = [model.layer4[-1]]

In [ ]:
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=True)

In [ ]:
transform = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

img = Image.open("cat.jpeg")

img_tensor = transform(img)
img_tensor = img_tensor.unsqueeze(0)
img_tensor.requires_grad = True
img_tensor = img_tensor.to(device)

In [ ]:
targets = [ClassifierOutputTarget(281)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=img_tensor, targets=targets)

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
inv_transform = T.Compose(
    [
        T.Normalize(
            mean=(-1 * np.array(mean) / np.array(std)).tolist(), std=(1 / np.array(std)).tolist()
        ),
    ]
)

In [ ]:
# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
rgb_img = inv_transform(img_tensor).cpu().squeeze().permute(1, 2, 0).detach().numpy()
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

In [ ]:
plt.imshow(visualization)

In [ ]:
# from pytorch_grad_cam import GradCAMPlusPlus

In [ ]:
cam = GradCAMPlusPlus(model=model, target_layers=target_layers, use_cuda=True)

In [ ]:
grayscale_cam = cam(input_tensor=img_tensor, targets=targets)

In [ ]:
# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
rgb_img = inv_transform(img_tensor).cpu().squeeze().permute(1, 2, 0).detach().numpy()
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

In [ ]:
plt.imshow(visualization)

In [ ]:
from captum.robust import FGSM, PGD

In [ ]:
transform = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

img = Image.open("cat.jpeg")

img_tensor = transform(img)
img_tensor = img_tensor.unsqueeze(0)
img_tensor.requires_grad = True
img_tensor = img_tensor.to(device)

In [ ]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
inv_transform = T.Compose(
    [
        T.Normalize(
            mean=(-1 * np.array(mean) / np.array(std)).tolist(), std=(1 / np.array(std)).tolist()
        ),
    ]
)

In [ ]:
def get_prediction(model, image: torch.Tensor):
    model = model.to(device)
    img_tensor = image.to(device)
    with torch.no_grad():
        output = model(img_tensor)
    output = F.softmax(output, dim=1)
    prediction_score, pred_label_idx = torch.topk(output, 1)

    pred_label_idx.squeeze_()
    predicted_label = categories[pred_label_idx.item()]

    return predicted_label, prediction_score.squeeze().item()

    # print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')

In [ ]:
# Get original prediction
pred, score = get_prediction(model, img_tensor)

In [ ]:
def image_show(img, pred):
    npimg = inv_transform(img).squeeze().permute(1, 2, 0).detach().numpy()
    plt.imshow(npimg)
    plt.title("prediction: %s" % pred)
    plt.show()

In [ ]:
image_show(img_tensor.cpu(), pred + " " + str(score))

In [ ]:
# Construct FGSM attacker
fgsm = FGSM(model, lower_bound=-1, upper_bound=1)
perturbed_image_fgsm = fgsm.perturb(img_tensor, epsilon=0.16, target=285)
new_pred_fgsm, score_fgsm = get_prediction(model, perturbed_image_fgsm)

In [ ]:
image_show(perturbed_image_fgsm.cpu(), new_pred_fgsm + " " + str(score_fgsm))

In [ ]:
pgd = PGD(
    model, torch.nn.CrossEntropyLoss(reduction="none"), lower_bound=-1, upper_bound=1
)  # construct the PGD attacker

perturbed_image_pgd = pgd.perturb(
    inputs=img_tensor,
    radius=0.13,
    step_size=0.02,
    step_num=7,
    target=torch.tensor([199]).to(device),
    targeted=True,
)
new_pred_pgd, score_pgd = get_prediction(model, perturbed_image_pgd)

In [ ]:
image_show(perturbed_image_pgd.cpu(), new_pred_pgd + " " + str(score_pgd))

In [ ]:
feature_mask = (
    torch.arange(64 * 7 * 7)
    .reshape(8 * 7, 8 * 7)
    .repeat_interleave(repeats=4, dim=1)
    .repeat_interleave(repeats=4, dim=0)
    .reshape(1, 1, 224, 224)
)
print(feature_mask)

In [ ]:
feature_mask.shape

In [ ]:
print(feature_mask.to(device))
print(img_tensor.device)

In [ ]:
from captum.attr import FeatureAblation

ablator = FeatureAblation(model)
attr = ablator.attribute(img_tensor, target=285, feature_mask=feature_mask.to(device))
# Choose single channel, all channels have same attribution scores
pixel_attr = attr[:, 0:1]

In [ ]:
def pixel_dropout(image, dropout_pixels):
    keep_pixels = image[0][0].numel() - int(dropout_pixels)
    vals, _ = torch.kthvalue(pixel_attr.flatten(), keep_pixels)
    return (pixel_attr < vals.item()) * image

In [ ]:
from captum.robust import MinParamPerturbation

min_pert_attr = MinParamPerturbation(
    forward_func=model,
    attack=pixel_dropout,
    arg_name="dropout_pixels",
    mode="linear",
    arg_min=0,
    arg_max=1024,
    arg_step=16,
    preproc_fn=None,
    apply_before_preproc=True,
)

In [ ]:
pixel_dropout_im, pixels_dropped = min_pert_attr.evaluate(
    img_tensor, target=285, perturbations_per_eval=10
)
print("Minimum Pixels Dropped:", pixels_dropped)

In [ ]:
new_pred_dropout, score_dropout = get_prediction(model, pixel_dropout_im)

In [ ]:
image_show(pixel_dropout_im.cpu(), new_pred_dropout + " " + str(score_dropout))

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
transform = A.Compose([
    A.Normalize(mean=channel_means, std=channel_stdevs)
    A.GaussNoise(p=0.5)
    # A.RandomBrightnessContrast(p=0.5),
    ToTensorV2(),
    
])

In [ ]:
img = Image.open("cat.jpeg")
img = np.array(img)

img_tensor = transform(image=img)["image"]
img_tensor = img_tensor.unsqueeze(0)
# img_tensor.requires_grad = True
img_tensor = img_tensor.to(device)

In [ ]:
print(type(img_tensor))

In [ ]:
plt.imshow(img_tensor.squeeze().permute(1, 2, 0).detach().cpu().numpy())

In [ ]:
output = model(img_tensor.float())
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)

pred_label_idx.squeeze_()
predicted_label = categories[pred_label_idx.item()]

print("pred=", predicted_label)
print("score=", prediction_score)
print("index=", pred_label_idx)

In [ ]:
print(model)